The data within the CSV is complex, we need to use the "VDOT_Crash_Data_Dictionary_2022.pdf" to manually encode the data.

In [1]:
# First lets load the csv file into a pandas dataframe
import pandas as pd

df = pd.read_csv('data/crash_data.csv')
df.head()

C:\Users\ithom\AppData\Local\Temp\ipykernel_19268\3926041423.py:4: DtypeWarning: Columns (47,49,51,53,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/crash_data.csv')


,OBJECTID,Document_Nbr,Crash_Year,Crash_Severity,Driver_VehicleNumber,Driver_InjuryType,Driver_Age,Driver_Gender,Driver_Action_Type_Cd,Driver_Airbag_Deployment,...,Ped_Action,Ped_Drink,Ped_Cond,Ped_Al_Test,Ped_Drug,Ped_Rflct,Numberoflane,Carspeedlimit,Local Case CD,Route or Street Name
0,1,100200959,2009,B,1,PDO,48,Male,1. No Improper Action,2. Not Deployed,...,1. Crossing At Intersection With Signal,1. No Drinking,1. No Defects,4. No Test,3. Unknown,2. No,4.0,35.0,20093440243,RT 7LEESBURG PIKE
1,2,93010657,2009,O,1;2,PDO;PDO,45;57,Male;Male,22. Disregarded Stop or Yield Sign;1. No Impro...,2. Not Deployed;2. Not Deployed,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,R-2385,WARWICK BLVD
2,3,92440184,2009,O,1;2,NA;PDO,47,Male;Male,34. Hit and Run;1. No Improper Action,2. Not Deployed;2. Not Deployed,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,P09040800226,1800 BLK E BRAMBELTON
3,4,92870200,2009,O,1,PDO,17,Female,1. No Improper Action,2. Not Deployed,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,55.0,NaN,I 66 EAST
4,5,93400162,2009,C,1;2,C;PDO,53;63,Female;Male,11. Did Not Have Right-of-Way;1. No Improper A...,1. Deployed - Front;3. n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200909150302,DISTRIBUTOR DRIVE


In [2]:
# Then we can remove all non-categorical data
df = df.select_dtypes(include=['object'])
df.head()

,Crash_Severity,Driver_VehicleNumber,Driver_InjuryType,Driver_Age,Driver_Gender,Driver_Action_Type_Cd,Driver_Airbag_Deployment,Driver_Alcohol_Test_Type_Cd,Driver_Condition_Type_Cd,Driver_Distraction_Type_Cd,...,PED_Age,Ped_Gender,Ped_Action,Ped_Drink,Ped_Cond,Ped_Al_Test,Ped_Drug,Ped_Rflct,Local Case CD,Route or Street Name
0,B,1,PDO,48,Male,1. No Improper Action,2. Not Deployed,4. No Test,1. No Defects,n/a,...,23,Male,1. Crossing At Intersection With Signal,1. No Drinking,1. No Defects,4. No Test,3. Unknown,2. No,20093440243,RT 7LEESBURG PIKE
1,O,1;2,PDO;PDO,45;57,Male;Male,22. Disregarded Stop or Yield Sign;1. No Impro...,2. Not Deployed;2. Not Deployed,n/a; n/a,1. No Defects;1. No Defects,n/a; n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R-2385,WARWICK BLVD
2,O,1;2,NA;PDO,47,Male;Male,34. Hit and Run;1. No Improper Action,2. Not Deployed;2. Not Deployed,n/a; n/a,9. Unknown;1. No Defects,7. Eyes Not on Road; n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P09040800226,1800 BLK E BRAMBELTON
3,O,1,PDO,17,Female,1. No Improper Action,2. Not Deployed,n/a,1. No Defects,n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I 66 EAST
4,C,1;2,C;PDO,53;63,Female;Male,11. Did Not Have Right-of-Way;1. No Improper A...,1. Deployed - Front;3. n/a,n/a; n/a,1. No Defects;1. No Defects,n/a; n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200909150302,DISTRIBUTOR DRIVE


In [3]:
def split_string(string):
    # If the string is empty or nan, return an empty list
    if string == "" or pd.isnull(string):
        return []
    # Break down each string into a list if there is a semicolon
    if ";" in str(string):
        parts = string.split(";")                
        return parts
    else:
        return [string]
    
split_string("l;l")

['l', 'l']

In [4]:
# def find_id(string):
#     # Remove all leading spaces
#     string = str(string).strip()
#     # If the first value is a number
#     if len(string) > 0 and string[0].isdigit():
#         # If the value right after is a .
#         if len(string) > 1 and string[1] == ".":
#             return string[0]

def find_id(string):
    # Remove all leading spaces
    string = str(string).strip()

    # Initialize an empty result to accumulate digits
    result = ""

    # Loop through each character until the first non-digit or period is encountered
    for char in string:
        if char.isdigit():
            result += char
        elif char == ".":
            # Return the accumulated result if a period follows the digits
            return result
        else:
            # If a non-digit, non-period character is encountered, exit
            break

    # If no valid id is found, return None
    return None

find_id("1. No Improper Action")

'1'

In [5]:
def clean_values(string):
    individual_strings: list = split_string(string)
    output: list = []
    for string in individual_strings:
        if find_id(string):
            output.append(find_id(string))
        else:
            output.append(string)
    return output
        
print(clean_values("1. No Improper Action; 2. No Improper Action"))
print(clean_values("PDO;PDO"))
print(clean_values("22. Disregarded Stop or Yield Sign;1. No Impro...	"))


['1', '2']
['PDO', 'PDO']
['22', '1']


In [6]:
# Apply clean_values to the dataframe
df = df.map(clean_values)
df.head()

,Crash_Severity,Driver_VehicleNumber,Driver_InjuryType,Driver_Age,Driver_Gender,Driver_Action_Type_Cd,Driver_Airbag_Deployment,Driver_Alcohol_Test_Type_Cd,Driver_Condition_Type_Cd,Driver_Distraction_Type_Cd,...,PED_Age,Ped_Gender,Ped_Action,Ped_Drink,Ped_Cond,Ped_Al_Test,Ped_Drug,Ped_Rflct,Local Case CD,Route or Street Name
0,[B],[1],[PDO],[48],[Male],[1],[2],[4],[1],[ n/a],...,[23],[Male],[1],[1],[1],[4],[3],[2],[20093440243],[RT 7LEESBURG PIKE]
1,[O],"[1, 2]","[PDO, PDO]","[45, 57]","[Male, Male]","[22, 1]","[2, 2]","[ n/a, n/a]","[1, 1]","[ n/a, n/a]",...,[],[],[],[],[],[],[],[],[R-2385],[WARWICK BLVD]
2,[O],"[1, 2]","[NA, PDO]",[47],"[Male, Male]","[34, 1]","[2, 2]","[ n/a, n/a]","[9, 1]","[7, n/a]",...,[],[],[],[],[],[],[],[],[P09040800226],[1800 BLK E BRAMBELTON]
3,[O],[1],[PDO],[17],[Female],[1],[2],[ n/a],[1],[ n/a],...,[],[],[],[],[],[],[],[],[],[I 66 EAST]
4,[C],"[1, 2]","[C, PDO]","[53, 63]","[Female, Male]","[11, 1]","[1, 3]","[ n/a, n/a]","[1, 1]","[ n/a, n/a]",...,[],[],[],[],[],[],[],[],[200909150302],[DISTRIBUTOR DRIVE]
